In [39]:
import os
import pandas as pd
import locale

locale.setlocale(locale.LC_NUMERIC, '')

#Inputs Consider moving all of this shit to the config file....
current_quarter = 'Q1'


source_location = os.path.join('..','..','data','interim')
target_location = os.path.join('..','..','data','processed')

#Data sources located in the interim data section
source_kpi           = 'scorecard_kpi.csv'
source_metrics       = 'scorecard_extracted_metrics.csv'
source_jurisdictions = 'scorecard_extracted_juris_metrics.csv'
target_jurisdictions = 'scorecard_key_value_jurisdictions.csv'


#File Paths... source & targets
source_jurisdictions_file_path = os.path.join(source_location, source_jurisdictions)
target_jurisdictions_file_path = os.path.join(target_location, target_jurisdictions)

##########################################################################
# Data & Structure I am going to need to reference later in order to pull out relevant metrics

Quarters = {'July':'Q1',
           'August':'Q1',
           'September':'Q1',
           'October':'Q2',
           'November':'Q2',
           'December':'Q2',
           'January':'Q3',
           'February':'Q3',
           'March':'Q3',
           'April':'Q4',
           'May':'Q4',
           'June':'Q4'}

######################################################################
# Let's go ahead and work out the jurisdiction metrics first

data = pd.read_csv(source_jurisdictions_file_path)
data['Quarter'] = data['Month'].replace(Quarters)
data_v2 = data.groupby(['Quarter','Metrics']).aggregate(sum)
data_v2 = data_v2.reset_index()

data_v3 = pd.melt(data_v2, id_vars=['Quarter','Metrics'], var_name = 'Region')

######################################################################
# Let's read in the data and begin processing

data_v3['key'] = "_" + data_v3['Region'] + "_" + data_v3['Metrics'] + "_"
data_v3['key'] = data_v3['key'].str.replace(' ','_')
data_v3['key'] = data_v3['key'].str.replace('Economic','con')
data_v4 = data_v3[data_v3['Quarter'] == current_quarter]

key_value_out = data_v4[['Region','Metrics','Quarter','key','value']]
key_value_out['value'] = key_value_out['value'].fillna(0)
key_value_out['value'] = key_value_out['value'].apply(int)
key_value_out['value'] = key_value_out['value'].apply(lambda x: '{0:,d}'.format(x))


key_value_out.to_csv(target_jurisdictions_file_path, index = False)


In [57]:
key_value_out[key_value_out['Region'] == "Pacific Grove"][['Region','Metrics','value']]

,Region,Metrics,value
160,Pacific Grove,Additional Person Room Nights,56
161,Pacific Grove,Booked Room Nights,0
162,Pacific Grove,Definite Bookings,0
163,Pacific Grove,Earned Media,"393,125"
164,Pacific Grove,Economic Impact of Additional PRN,"18,648"
165,Pacific Grove,Economic Impact of Bookings,0
166,Pacific Grove,Economic Impact of Infl room nights**,"960,372"
167,Pacific Grove,Economic Impact of RN booked thru web,"17,649"
168,Pacific Grove,Group Business Leads,41
169,Pacific Grove,Group Lead Room Nights,"8,592"
